![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Assertion Status Models

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.0
sparknlp_jsl.version : 3.4.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.7:7077 
 AppName 
 Databricks Shell

# Clinical Assertion Model (with pretrained models)

## Pretrained `assertion_dl` model

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
  .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
  .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][OK!]

In [0]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

Out[4]: '\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") .setInputCols(["sentence", "ner_chunk", "embeddings"]) .setOutputCol("assertion")\n'

In [0]:
import pandas as pd

text = 'Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain'

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain
Out[4]:

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,anxious,PROBLEM,conditional
2,alopecia,PROBLEM,absent
3,pain,PROBLEM,absent


In [0]:
from sparknlp_display import AssertionVisualizer

visualizer = AssertionVisualizer()

vis = visualizer.display(light_result, 'ner_chunk', 'assertion', return_html=True)
#visualizer.set_label_colors({'TREATMENT':'#008080', 'PROBLEM':'#800080'})


displayHTML(vis)

Patient has a headache PROBLEM present for the last 2 weeks and appears anxious PROBLEM conditional when she walks fast. No alopecia PROBLEM absent noted. She denies pain PROBLEM absent

In [0]:
! wget -q	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv -P /dbfs/

In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"""Monomeric sarcosine oxidase (MSOX) is a flav...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| assertion|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
A numerical metho...|[{document, 0, 88...|[{document, 0, 15...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 69, 95, ...|[{assertion, 69, ...|
Recording of the ...|[{document, 0, 44...|[{document, 0, 10...|[{token, 0, 8, Re...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 30, ...|[{assertion, 13, ...|
Colicin E5 cleave...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 6, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, Co...|[{assertion, 0, 9...|
Recombinant adeno...|[{document, 0, 13...|[{document, 0, 20...|[{token, 0, 10, R...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 28, R...|[{assertion, 0, 2...|
To understand its...|[{document, 0, 29...|[{document, 0, 19...|[{token, 0, 1, To...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 37, 52, ...|[{assertion, 37, ...|
OBJECTIVE: To rev...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 63, 91, ...|[{assertion, 63, ...|
AIM: Understandin...|[{document, 0, 15...|[{document, 0, 16...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 46, 74, ...|[{assertion, 46, ...|
In the present st...|[{document, 0, 92...|[{document, 0, 21...|[{token, 0, 1, In...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 19, t...|[{assertion, 3, 1...|
AIM: To describe ...|[{document, 0, 14...|[{document, 0, 20...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 17, 32, ...|[{assertion, 17, ...|
Although the risk...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 7, Al...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 118, 163...|[{assertion, 118,...|
Successful long-t...|[{document, 0, 16...|[{document, 0, 12...|[{token, 0, 9, Su...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 43, S...|[{assertion, 0, 4...|
Pulmonary alveola...|[{document, 0, 16...|[{document, 0, 15...|[{token, 0, 8, Pu...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 94, ...|[{assertion, 68, ...|
Prenatal care is ...|[{document, 0, 26...|[{document, 0, 68...|[{token, 0, 7, Pr...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 70, 86, ...|[{assertion, 70, ...|
Controversy exist...|[{document, 0, 12...|[{document, 0, 18...|[{token, 0, 10, C...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 43, 66, ...|[{assertion, 43, ...|
Children with a f...|[{document, 0, 34...|[{document, 0, 13...|[{token, 0, 7, Ch...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 34, 46, ...|[{assertion, 34, ...|
The mucosal immun...|[{document, 0, 19...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 51, 57, ...|[{assertion, 51, ...|
Urinary concretio...|[{document, 0, 86...|[{document, 0, 15...|[{token, 0, 6, Ur...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 18, U...|[{assertion, 0, 1...|
In this study, th...|[{document, 0, 14...|[{document, 0, 17...|[{token, 0, 1, In...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 12, t...|[{assertion, 3, 1...|
The presence of c...|[{document, 0, 44...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 16, 32, ...|[{assertion, 16, ...|
Phosphatidylethan...|[{document, 0, 17...|[{document, 0, 26...|[{token, 0, 23, P...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 48, P...|[{assertion, 0, 4...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('sentence.result').take(1)

Out[14]: [Row(result=['A numerical method was developed using geometrical optics to predict far-fieldoptical scattering from particles that are symmetric about the optic axis.', 'Thediffractive component of scattering is calculated and combined with thereflective and refractive components to give the total scattering pattern.', 'Thephase terms of the scattered light are calculated as well.', 'Verification of themethod was achieved by assuming a spherical particle and comparing the results toMie scattering theory.', 'Agreement with the Mie theory was excellent in theforward-scattering direction.', 'However, small-amplitude oscillations near therainbow regions were not observed using the numerical method.', 'Numerical data fromspheroidal particles and hemispherical particles are also presented.', 'The use ofhemispherical particles as a calibration standard for intensity-type opticalparticle-sizing instruments is discussed.'])]

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,  result.ner_chunk.begin, result.ner_chunk.end, result.ner_chunk.metadata, 
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['sentence']").alias("sent_id"),
              F.expr("cols['4']").alias("assertion"),).show(truncate=False)



+-----------------------------------------------------------+-----+---+---------+-------+-----------+
chunk |begin|end|ner_label|sent_id|assertion |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
the G-protein-activated inwardly rectifying potassium (GIRK|48 |106|TREATMENT|0 |conditional|
the genomicorganization |142 |164|TREATMENT|1 |present |
a candidate gene forType II diabetes mellitus |210 |254|PROBLEM |1 |present |
byapproximately |380 |394|TREATMENT|2 |present |
single nucleotide polymorphisms |464 |494|TREATMENT|3 |present |
aVal366Ala substitution |532 |554|PROBLEM |3 |present |
an 8 base-pair |561 |574|PROBLEM |3 |present |
insertion/deletion |581 |598|PROBLEM |4 |present |
Ourexpression studies |601 |621|TEST |5 |present |
the transcript in various humantissues |648 |685|PROBLEM |5 |present |
fat andskeletal muscle |749 |770|PROBLEM |5 |possible |
furtherstudies |830 |843|PROBLEM |6 |present |
the KCNJ9 protein |864 |880|TREATMENT|6 |present |
evaluation |892 |901|TEST |6 |possible |
Type II diabetes |940 |955|PROBLEM |6 |present |
the treatment |67 |79 |TREATMENT|0 |present |
breast cancer |84 |96 |PROBLEM |0 |present |
the standard therapy |109 |128|TREATMENT|0 |present |
anthracyclines |167 |180|TREATMENT|0 |present |
taxanes |186 |192|TREATMENT|0 |present |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
only showing top 20 rows

## Pretrained `assertion_dl_radiology` model

In [0]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model for radiology
radiology_ner = MedicalNerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
# coming from sparknlp_jsl.annotator !!

radiology_assertion = AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    radiology_ner,
    ner_converter,
    radiology_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_radiology download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]
assertion_dl_radiology download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][ — ][OK!]

In [0]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

In [0]:
data = spark.createDataFrame([[text]]).toDF("text")

In [0]:
result = radiologyAssertion_model.transform(data)

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata, result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------------------+---------------+-------+---------+
chunk |ner_label |sent_id|assertion|
+-------------------+---------------+-------+---------+
effusion |ImagingFindings|0 |Negative |
pneumothorax |ImagingFindings|0 |Negative |
displaced fractures|ImagingFindings|0 |Confirmed|
+-------------------+---------------+-------+---------+

## Writing a generic Assertion + NER function

In [0]:
from pyspark.sql.functions import monotonically_increasing_id


def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

    sentenceDetector = SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

    tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
    word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    base_pipeline = Pipeline(stages=[
                        documentAssembler,
                        sentenceDetector,
                        tokenizer,
                        word_embeddings])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")
      

    base_model = get_base_pipeline (embeddings)

    nlpPipeline = Pipeline(stages=[
        base_model,
        loaded_ner_model,
        ner_converter,
        clinical_assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata, result.assertion.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label"),
                              F.expr("cols['2']").alias("assertion"))\
                              .filter("ner_label!='O'")

    return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 1000

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+-----------+
 chunk|ner_label| assertion|
+--------------------+---------+-----------+
the G-protein-act...|TREATMENT|conditional|
the genomicorgani...|TREATMENT| present|
a candidate gene ...| PROBLEM| present|
 byapproximately|TREATMENT| present|
single nucleotide...|TREATMENT| present|
aVal366Ala substi...| PROBLEM| present|
 an 8 base-pair| PROBLEM| present|
 insertion/deletion| PROBLEM| present|
Ourexpression stu...| TEST| present|
the transcript in...| PROBLEM| present|
fat andskeletal m...| PROBLEM| possible|
 furtherstudies| PROBLEM| present|
 the KCNJ9 protein|TREATMENT| present|
 evaluation| TEST| possible|
 Type II diabetes| PROBLEM| present|
 the treatment|TREATMENT| present|
 breast cancer| PROBLEM| present|
the standard therapy|TREATMENT| present|
 anthracyclines|TREATMENT| present|
 taxanes|TREATMENT| present|
+--------------------+---------+-----------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+---------+---------+
 chunk|ner_label|assertion|
+----------------+---------+---------+
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 vinorelbine| DRUG| present|
 vinorelbine| DRUG| present|
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 Vinorelbinewas| DRUG| absent|
 25 mg/m(2| STRENGTH| present|
 intravenously| ROUTE| absent|
 on days|FREQUENCY| present|
 thatvinorelbine| DRUG| present|
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 DOX| DRUG| absent|
 trandolapril| DRUG| possible|
 losartan| DRUG| present|
 3-week| DURATION| absent|
 placebo| DRUG| present|
2 mgtrandolapril| STRENGTH| absent|
 50 mg| STRENGTH| possible|
+----------------+---------+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+---------+---------+
 chunk|ner_label|assertion|
+----------------+---------+---------+
capsule of Advil| DRUG| absent|
+----------------+---------+---------+

In [0]:
import pandas as pd

def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [0]:
clinical_text = """
Patient with severe fever and sore throat. 
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

get_clinical_assertion_light (light_model, clinical_text)


Out[31]:

,chunks,entities,assertion
0,severe fever,PROBLEM,present
1,sore throat,PROBLEM,present
2,stomach pain,PROBLEM,absent
3,an epidural,TREATMENT,present
4,PCA,TREATMENT,present
5,pain control,PROBLEM,present
6,short of breath,PROBLEM,conditional
7,CT,TEST,present
8,lung tumor,PROBLEM,present
9,Alzheimer,PROBLEM,associated_with_someone_else


## Assertion with BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+----------------------+---------+
chunk |ner_label |assertion|
+----------------+----------------------+---------+
optic axis |Multi-tissue_structure|possible |
esophageal ECG |Multi-tissue_structure|present |
nose |Organ |present |
esophageal ECG |Multi-tissue_structure|present |
E5 |Gene_or_gene_product |present |
Tyr |Amino_acid |present |
Asn |Amino_acid |present |
Asp |Amino_acid |present |
Escherichia coli|Organism |present |
RNases Aand T1 |Gene_or_gene_product |present |
E5-CRD |Gene_or_gene_product |absent |
usualRNases |Gene_or_gene_product |absent |
RNase |Gene_or_gene_product |present |
Lys25 |Simple_chemical |present |
Gln29 |Simple_chemical |present |
Lys60 |Simple_chemical |present |
Ile94 |Gene_or_gene_product |present |
Arg33 |Gene_or_gene_product |possible |
Arg33 |Gene_or_gene_product |possible |
Lys60 |Simple_chemical |present |
+----------------+----------------------+---------+
only showing top 20 rows

# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like present or conditional; and do not want absent conditions get out of your pipeline.

In [0]:
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

assertion_filterer = AssertionFilterer()\
      .setInputCols("sentence","ner_chunk","assertion")\
      .setOutputCol("assertion_filtered")\
      .setWhiteList(["present"])

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      clinical_assertion,
      assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

In [0]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

Out[34]: dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

Out[35]: [('a headache', 'present'),
 ('a head CT', 'present'),
 ('anxious', 'present'),
 ('fast', 'present'),
 ('Alopecia', 'present'),
 ('pain', 'absent')]

In [0]:
assertion_filterer.getWhiteList()

Out[36]: ['present']

In [0]:
light_result['assertion_filtered']

Out[37]: ['a headache', 'a head CT', 'anxious', 'fast', 'Alopecia']

# Train a custom Assertion Model

**WARNING:** For training an Assertion model, please use TensorFlow version 2.3

In [0]:
!wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv -P /dbfs/

In [0]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/i2b2_assertion_sample.csv")


In [0]:
assertion_df.show(3, truncate=100)

+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
 text| target| label|start|end|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
Due to dye load required to assess the coronary disease were not visualized .|the coronary disease|present| 7| 9|
 She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .| undesired fertility|present| 1| 2|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
only showing top 3 rows

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 814
Test Dataset Count: 186

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)


+----------------------------+-----+
label |count|
+----------------------------+-----+
present |546 |
absent |182 |
hypothetical |41 |
possible |34 |
associated_with_someone_else|6 |
conditional |5 |
+----------------------------+-----+

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

#### With a custom tf graph

In [0]:
import tensorflow
from sparknlp_jsl.training import tf_graph

In [0]:
%fs mkdirs file:/dbfs/assertion_tf_graphs

res2: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/assertion_logs

res3: Boolean = true

In [0]:
%fs ls file:/dbfs

path,name,size
file:/dbfs/AntBNC_lemmas_ver_001.txt,AntBNC_lemmas_ver_001.txt,1348552
file:/dbfs/AskAPatient.fold-0.test.txt,AskAPatient.fold-0.test.txt,33834
file:/dbfs/AskAPatient.fold-0.train.txt,AskAPatient.fold-0.train.txt,594794
file:/dbfs/AskAPatient.fold-0.validation.txt,AskAPatient.fold-0.validation.txt,32162
file:/dbfs/FileStore/,FileStore/,4096
file:/dbfs/JohnSnowLabs/,JohnSnowLabs/,4096
file:/dbfs/MT_00.pdf,MT_00.pdf,528022
file:/dbfs/MT_01.pdf,MT_01.pdf,165397
file:/dbfs/MT_02.pdf,MT_02.pdf,309334
file:/dbfs/MT_03.pdf,MT_03.pdf,248334


In [0]:
tf_graph.build("assertion_dl",build_params={"n_classes": 6}, model_location="/dbfs/assertion_tf_graphs", model_filename="auto")

/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/assertion_dl/assertion_model.py:48: BasicLSTMCell.__init__ (from tensorflow.python.keras.layers.legacy_rnn.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/rnn.py:229: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/rnn.py:438: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:738: Layer.add_variable (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:744: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
assertion_dl graph exported to /dbfs/assertion_tf_graphs/blstm_34_32_30_200_6.pb

In [0]:
%fs ls file:/dbfs/assertion_tf_graphs/

path,name,size
file:/dbfs/assertion_tf_graphs/blstm_34_32_30_200_6.pb,blstm_34_32_30_200_6.pb,1333335


In [0]:
assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(50)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('dbfs:/assertion_logs')\
    .setGraphFolder('dbfs:/assertion_tf_graphs')


clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertionStatus])

In [0]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

In [0]:
assertion_model.stages

Out[27]: [DocumentAssembler_3912fb43d16f,
 Doc2Chunk_7659725cd03d,
 REGEX_TOKENIZER_d68f222cda3d,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 ASSERTION_DL_f46aadd1b4c0]

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+------------+--------------+
 label| result|
+------------+--------------+
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
hypothetical|[hypothetical]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
 present| [present]|
+------------+--------------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()

In [0]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['label']))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 absent 0.61 0.74 0.67 38
associated_with_someone_else 0.00 0.00 0.00 0
 conditional 0.00 0.00 0.00 0
 hypothetical 0.56 0.62 0.59 8
 possible 0.00 0.00 0.00 0
 present 0.92 0.78 0.84 140

 accuracy 0.76 186
 macro avg 0.35 0.36 0.35 186
 weighted avg 0.84 0.76 0.80 186

In [0]:
#saving the model that we've trained
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/custom_assertion_model')